# Untersuchung der Entwicklung von Verbraucherpreisen und Energiekosten 2023


## Übersicht


Verbraucherpreise sind während der Pandemie und insbesondere nach dem  Ukraine-Krieg stark gestiegen. Die prozentuale Veränderung gegenüber dem Vorjahr bildet dabei die Inflationsrate. Hauptsächlich werden die Energiekosten dafür verantwortlich gemacht. Nun wollen wir die Entwicklung des Verbraucherpreises und der Erzeugerpreise untersuchen.

**Mögliche Fragen:**
1. Wie stark sind die Preise gestiegen?
2. Wann sind die Preise gestiegen?
3. Welche Produkte sind besonders betroffen?
4. Ist der Anstieg an Verbraucherpreisen und Erzeugerpreisen proportional?

## Daten

https://www.bundesbank.de/de/statistiken/konjunktur-und-preise/erzeuger-und-verbraucherpreise/erzeuger-und-verbraucherpreise-775284

Die Daten für die Preisentwicklung in Deutschland wird, mithilfe von Messgrößen, von dem Statistischen Bundesamt erhoben. Der **Verbraucherpreisindex (VPI)** misst die prozentuale Veränderung der Preise bestimmter Waren und Dienstleistungen für private Haushalte. Die Veränderung des VPI zum Vorjahr bildet die Inflationsrate. Der **Erzeugerpreisindex (EPI)**  

## Data Cleaning

### VPI

Die Verbraucherindex-Daten von 2018-2023 (Basisjahr=2020) finden wir auf folgender Seite des statistischen Bundesamtes: https://www.destatis.de/DE/Themen/Wirtschaft/Preise/Verbraucherpreisindex/Tabellen/Verbraucherpreise-12Kategorien.html 

Die monatlichen Indizes laden wir in **Power BI**.


<img src='Bilder/2-1-VPI_load.png' width="600"/>

Um sie zu transformieren öffnen wir sie in **PowerQuery**. Wir stellen fest, dass für die zweite Tabelle, aufgrund der zusätzlichen Notiz, falsche column header verwendet wurden. Wir übernehmen die erste Zeile als header und entfernen die zweite Zeile.


<img src='Bilder/2-2-VPI_Table1.png' width="1000"/>

Mithilfe der *fill* Funktion übernehmen wir die korrekten Jahreszahlen für die null-Werte. Wir erstellen mit *Add Column From Examples* eine neue Spalte in dem Format "Jahr Monat" und verknüpfen die Tabellen mit einem *INNER JOIN* auf der neuen Spalte.

<img src='Bilder/2-3-VPI_merge_date_columns.png' width="1000"/>

Wir korrigieren die Spaltennamen und Spalten-Datentypen und bringen mit der Funktion *Unpivot* die Tabelle von der wide form in die long form.


<img src='Bilder/2-4-VPI_unpivot.png' width="1000"/>

<img src='Bilder/2-5-EPI_transform.png' width="1000"/>

### EPI gewerbliche Produkte

Quelle: https://www.destatis.de/DE/Themen/Wirtschaft/Preise/Erzeugerpreisindex-gewerbliche-Produkte/_inhalt.html#238942

Dieses mal benutzen wir **Python** für das Data Cleaning. Dazu laden wir die entsprechende Tabelle als Excel Datei herunter. 

Wir laden die Daten in ein Pandas Dataframe.

In [1]:
import pandas as pd

file_path = '1-EPI-Gewerbliche-Produkte.xlsx'
df = pd.read_excel(file_path)
df.head(8)

,"Producer price index for industrial products: Germany,\nmonths, product classification (GP2009 2-/3-/4-/5-/6-/\n9-digit codes/special items)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73
0,Index of producer prices of industrial products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Producer price index for industrial products (...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GP2009 (2-digit codes): Industrial products,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,January,February,March,April,May,June,July,August,...,March,April,May,June,July,August,September,October,November,December
5,GP09-05,Kohle,97.3,97.3,98.9,98.4,98.4,98.4,98.3,98.3,...,141,141,143,148.6,...,...,...,...,...,...
6,GP09-06,Erdöl und Erdgas,97.6,97,96,99.1,101,105.2,109.5,108.8,...,252.8,217.5,198.6,171.6,...,...,...,...,...,...
7,GP09-08,"Steine und Erden, sonstige Bergbauerzeugnisse",103.9,104,104,104.3,105,104.9,105.2,105,...,152.1,152.5,153.1,153.6,...,...,...,...,...,...


Wir Erstellen eine Funktion um die Spalten-Namen zu bestimmen.

In [2]:
def column_names():
    # Name der ersten Spalte 
    columns = ['Kategorie']
    # Zeitraum in der Form: 'JJJJ-MM' 
    for year in range(2018, 2024):
        for month in range(1, 13):
            columns.append(f'{year}-{month:02d}')
    return columns

Wir löschen die Reihen mit falschen und unnötigen Daten, wandeln die Daten in den long-Format und verändern die Datentypen

In [3]:
def clean_data(df):
    # Drop rows with missing data in column: 'Unnamed: 1'
    df = df.dropna(subset=['Unnamed: 1'])
    # Drop column: 'Producer price index for industrial products: Germany, months, product classification (GP2009 2-/3-/4-/5-/6-/ 9-digit codes/special items)'
    df = df.drop(columns=['Producer price index for industrial products: Germany,\nmonths, product classification (GP2009 2-/3-/4-/5-/6-/\n9-digit codes/special items)'])
    # Rename columns
    df.columns = column_names()
    # Drop Month with no data
    columns_to_drop = df.columns[df.eq('...').any()]
    df.drop(columns=columns_to_drop, inplace=True)
    
    # Pivot into long format
    df = pd.melt(df, id_vars='Kategorie', var_name='Datum', value_name='EPI')
    
    # Change column type to float16 for column: 'EPI'
    df = df.astype({'EPI': 'float16'})
    # Change column type to datetime64[ns] for column: 'Datum'
    df = df.astype({'Datum': 'datetime64[ns]'})
    # Change column type to string for column: 'Kategorie'
    df = df.astype({'Kategorie': 'category'})
    
    # Reset Index
    df = df.reset_index(drop=True)
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,Kategorie,Datum,EPI
0,Kohle,2018-01-01,97.3125
1,Erdöl und Erdgas,2018-01-01,97.6250
2,"Steine und Erden, sonstige Bergbauerzeugnisse",2018-01-01,103.8750
3,Nahrungsmittel und Futtermittel,2018-01-01,104.5000
4,Getränke,2018-01-01,103.5000


Schließlich speichern wir die gesäuberten Daten in eine Excel Datei.

In [4]:
df_clean.to_excel('1-EPI-Gewerbliche-Produkte-clean.xlsx', sheet_name='EPI', index=False)

### EPI Dienstleistungen

Quelle: https://www.destatis.de/DE/Themen/Wirtschaft/Preise/Erzeugerpreisindex-Dienstleistungen/_inhalt.html#

Die Daten für das EPI von Dienstleistungen ist nur in Quartalen verfügbar. Wir benutzen, wie bei dem VPI, Power Query. Um die Spalten in die richtige Form zu bringen, verwenden wir ein Python script. Die restlichen Schritte sind vergleichbar mit den Vorherigen Queries. 

In [ ]:
# Das verwendete Python script

# 'dataset' holds the input data for this script
import pandas as pd
df = dataset
column_names = ['Code','Kategorie']
# Zeitraum in der Form: 'JJJJ-QTR' 
for year in range(2018, 2024):
    for qtr in range(1, 5):
        column_names.append(f'{year}-{qtr:02d}')
df.columns = column_names

<img src='Bilder/2-6-EPI_Dienstleistungen.png' width="1000"/>

## Data Exploration

**Mögliche Fragen:**
1. Wie stark sind die Preise gestiegen?
2. Wann sind die Preise gestiegen?
3. Welche Produkte sind besonders betroffen?
4. Ist der Anstieg an Verbraucherpreisen und Erzeugerpreisen proportional?

Basis Jahr für VPI ist 2020. Also beträgt der Index im Jahr 2020 100.

Basis Jahr für EPI ist 2015. Also beträgt der Index im Jahr 2015 100.

Vor der Pandemie lag die Inflationsrate für das Jahr 2019 bei 1,42%. Im März 2020 erklärt die Weltgesundheitsorganisation (WHO) die momentane Lage als weltweite Pandemie. Die Inflationsrate nimmt ab und vom Juli bis Oktober herrscht sogar eine deflationäre Phase. Besonders betroffen sind die Preise für "Verkehr", "Freizeit, Unterhaltung und Kultur"und "Bekleidung und Schuhe". Der minimale Wert lag im Juli 2020 für "Freizeit, Unterhaltung und Kultur" und betrug -5,87%. Die Inflationsrate für das gesamte Jahr 2020 betrug 0,54%.

<img src='Bilder/3-1-VPI_Inflation.png' width="1000"/>

<!-- <img src='Bilder/3-2-VPI_Deflation_Pandemie.png' width="1000"/> -->

Ab 2021 steigt die Inflationsrate an. Besonders betroffen ist die Produktgruppe "Verkehr". So ist die Inflationsrate für "Verkehr" im Juli 2021 9,04% und der Index beträgt 108,6. Nach dem Angriff auf die Ukraine steigert sich die Inflationsrate rasant. Im Februar liegt die Inflation bei 4,33%, nach dem Start der Invasion, im März schon bei 5,88%. Der Höhepunkt für die Inflationsrate liegt im Oktober 2022 bei 8,82%. Der Höhepunkt für den VPI liegt im Juni 2023 bei 116,8.

<img src='Bilder/3-3-VPI_index.png' width="1000"/>

Nach dem Angriff auf die Ukraine sind "Nahrungsmittel und alkoholfreie Getränke" die am größten betroffene Produkte. Im März 2023 beträgt die Inflationsrate für diese Produktgruppe bei 21,20% und der VPI bei 131,5, d.h. dass "Nahrungsmittel und alkoholfreie Getränke" 31,5% teurer sind als im Basisjahr 2020.

<img src='Bilder/3-4-VPI_Vergleich.png' width="1000"/>

Der Verlauf des EPI ist ähnlich zum Verlauf des VPI jedoch sind die Werte für die Inflationsrate erheblich größer. So Liegt die Inflationsrate für den August 2022 bei 45,85% und im spezifischen für "Energie" bei 138,98%.

<img src='Bilder/3-5-EPI_max.png' width="1000"/>

Die Produktgruppen, die am frühesten von der Inflation betroffen waren, sind "Erdöl und Erdgas", "Kokerzeugnisse und Mineralölerzeugnisse" und "Metalle". 

<img src='Bilder/3-6-EPI_Güter1.png' width="1000"/>

Im Vergleich zu anderen Produkten stechen Nahrungsmittel heraus, weil der VPI stärker gestiegen ist als vieleicht zu vermuten wäre.

<img src='Bilder/3-7-EPI_Güter2.png' width="1000"/>

Der EPI für Dienstleistungen hingegen ist im Vergleich viel weniger von der Inflation betroffen. Güter- und Personenbeförderung in der Schiffsfahrt sind hier die Außnahme. 

<img src='Bilder/3-8-EPI_Dienstleistungen.png' width="1000"/>

### Fazit

* Sehr stark abhängig von Energiepreisen 
    * Industriegüter besonders betroffen
* Nahrungsmittel im Vergleich zu anderen Produktgruppen größere Inflation im Vergleich zum EPI
    * Frage: Wieso?
* Inflation Energiepreise schon vor dem Angriff auf die Ukraine
    * Anstieg an globalen Ölpreisen.
    * Diversifikation nötig
* Energiepreise wieder gesunken
* VPI und EPI jedoch noch nicht 